# 数据处理的三个阶段为：数据准备、数据转换、数据聚合

## 1、数据准备：加载、组装（合并、拼接、组合）、变形（轴向旋转）、删除


### 1.1 组装merge（）（即数据关联匹配）

合并:pandas.mege():根据一个或多个键连接多行，类似于sql的join操作（数据关联匹配）

拼接：pandas.concat():按照轴把多个对象拼接起来

结合：pandas.DataFrame.combine_first():从另外一个数据结构获取数据，连接重合的数据，以填充缺失值

In [38]:
import numpy as np
import pandas as pd
frame1=pd.DataFrame({'id':['ball','pencil','pen','mug','ashtray'],
                    'price':[12.22,31.20,33.31,10.30,23.00],
                    'brand':['OME','DE','LV','ER','DQ']})
frame1

,brand,id,price
0,OME,ball,12.22
1,DE,pencil,31.20
2,LV,pen,33.31
3,ER,mug,10.30
4,DQ,ashtray,23.00


In [8]:
frame2=pd.DataFrame({'id':['pencil','pencil','ball','pen'],
                    'color':['white','red','red','black'],
                    'brand':['OME','PID','DQ','LV']})
frame2

,brand,color,id
0,OME,white,pencil
1,PID,red,pencil
2,DQ,red,ball
3,LV,black,pen


In [10]:
pd.merge(frame1,frame2) # 左侧：id，brand与右侧：id，brand内关联
# 返回的DataFrame对象由原来两个DataFrame对象中ID相同的行组成，除了id这一列，
# 新dataframe包含原来分属于两个dataframe的其他列

,brand,id,price,color
0,LV,pen,33.31,black


##### 若两个DataFrame基准列的名称相同， on选项指定合并操作所依据的基准列

In [11]:
pd.merge(frame1,frame2,on='id')  

,brand_x,id,price,brand_y,color
0,OME,ball,12.22,DQ,red
1,DE,pencil,31.20,OME,white
2,DE,pencil,31.20,PID,red
3,LV,pen,33.31,LV,black


##### 若两个DataFrame基准列的名称不相同，用left_on和right_on选项指定第一个和第二个DataFrame的基准列

In [12]:
frame2.columns=['brand','color','sid']
frame2

,brand,color,sid
0,OME,white,pencil
1,PID,red,pencil
2,DQ,red,ball
3,LV,black,pen


In [13]:
pd.merge(frame1,frame2,left_on='id',right_on='sid')

,brand_x,id,price,brand_y,color,sid
0,OME,ball,12.22,DQ,red,ball
1,DE,pencil,31.20,OME,white,pencil
2,DE,pencil,31.20,PID,red,pencil
3,LV,pen,33.31,LV,black,pen


In [15]:
pd.merge(frame1,frame2,left_on=['brand','id'],right_on=['brand','sid'])
# 以多列为基准项进行合并

,brand,id,price,color,sid
0,LV,pen,33.31,black,pen


##### 连接方式：默认为内连接（即内关联），外连接：how=‘outer’，左连接：how=‘left’，右连接：how=‘right’

In [21]:
# 外关联：保留全部数据
pd.merge(frame1,frame2,left_on='id',right_on='sid',how='outer')

,brand_x,id,price,brand_y,color,sid
0,OME,ball,12.22,DQ,red,ball
1,DE,pencil,31.20,OME,white,pencil
2,DE,pencil,31.20,PID,red,pencil
3,LV,pen,33.31,LV,black,pen
4,ER,mug,10.30,NaN,NaN,NaN
5,DQ,ashtray,23.00,NaN,NaN,NaN


In [20]:
#  右关联：保留全部右数据源，左数据源保留匹配后的行
pd.merge(frame1,frame2,left_on='id',right_on='sid',how='right')

,brand_x,id,price,brand_y,color,sid
0,OME,ball,12.22,DQ,red,ball
1,DE,pencil,31.20,OME,white,pencil
2,DE,pencil,31.20,PID,red,pencil
3,LV,pen,33.31,LV,black,pen


### 1.2 拼接：concatenate（）（即行追加、列追加）
concat()函数默认按照axis=0表示行追加，axis=1表示列追加

join=‘inner’：执行内连接操作，默认为外连接

In [29]:
array1=pd.DataFrame(np.arange(9).reshape(3,3))
array1

,0,1,2
0,0,1,2
1,3,4,5
2,6,7,8


In [28]:
array2=pd.DataFrame(np.arange(11,20).reshape(3,3))
array2

,0,1,2
0,11,12,13
1,14,15,16
2,17,18,19


In [31]:
pd.concat([array1,array2],axis=1)  # axis=1列追加

,0,1,2,0,1,2
0,0,1,2,11,12,13
1,3,4,5,14,15,16
2,6,7,8,17,18,19


In [33]:
pd.concat([array1,array2],axis=0)  #axis=0行追加

,0,1,2
0,0,1,2
1,3,4,5
2,6,7,8
0,11,12,13
1,14,15,16
2,17,18,19


### 1.3 组合 combine_first()
combine_first()；用来组合series对象，同时对其数据。使用环境：两个数据集的索引完全或部分重合

In [36]:
ser1=pd.Series(np.random.rand(5),index=[1,2,3,4,5])
ser1

1    0.007468
2    0.266148
3    0.786905
4    0.844003
5    0.884407
dtype: float64

In [37]:
ser2=pd.Series(np.random.rand(4),index=[2,3,4,6])
ser2

2    0.042459
3    0.330524
4    0.596039
6    0.425764
dtype: float64

In [39]:
ser1.combine_first(ser2)

1    0.007468
2    0.266148
3    0.786905
4    0.844003
5    0.884407
6    0.425764
dtype: float64

In [40]:
ser2.combine_first(ser1)

1    0.007468
2    0.042459
3    0.330524
4    0.596039
5    0.884407
6    0.425764
dtype: float64

### 1.4 变形（轴向旋转）（即列转行或行转列）stack()  & unstack()
a、按等级索引旋转：轴向旋转有两个基础操作：

入栈（stacking）：旋转数据结构，把列转换为行。

出栈（unstacking）：把行转换为列

In [41]:
frame3=pd.DataFrame(np.arange(9).reshape(3,3),
                    index=['white','black','red'],
                   columns=['ball','pen','pencil'])
frame3

,ball,pen,pencil
white,0,1,2
black,3,4,5
red,6,7,8


In [43]:
# 对dataframe对象应用unstack（）函数，把行转换为列，从而得到一个series对象
frame3.unstack()

ball    white    0
        black    3
        red      6
pen     white    1
        black    4
        red      7
pencil  white    2
        black    5
        red      8
dtype: int32

In [42]:
# 对dataframe对象应用stack（）函数，把列转换为行，从而得到一个series对象
frame3.stack()

white  ball      0
       pen       1
       pencil    2
black  ball      3
       pen       4
       pencil    5
red    ball      6
       pen       7
       pencil    8
dtype: int32

### 插播：层次化索引(等级索引)
层次化索引：指能在一个数组上拥有多个索引（单条轴可以有多级索引）

##### 等级索引的series对象

In [82]:
# 创建一个包含两层的数据结构  （多重索引的series）
data=pd.Series(np.arange(9),
              index=[['white','white','white','black','black','black','red','red','red'],
                    ['ball','pen','pencil','ball','pen','pencil','ball','pen','pencil']])
data

white  ball      0
       pen       1
       pencil    2
black  ball      3
       pen       4
       pencil    5
red    ball      6
       pen       7
       pencil    8
dtype: int32

In [83]:
data.index

MultiIndex(levels=[['black', 'red', 'white'], ['ball', 'pen', 'pencil']],
           labels=[[2, 2, 2, 0, 0, 0, 1, 1, 1], [0, 1, 2, 0, 1, 2, 0, 1, 2]])

In [84]:
# 选取第一列索引中某一索引的元素
data[['white']]

white  ball      0
       pen       1
       pencil    2
dtype: int32

In [85]:
# 选取第二列索引中某一索引项的元素
data[:,'pen']

white    1
black    4
red      7
dtype: int32

In [86]:
# 选取某一特定的元素，指定两个索引
data['red','ball']

6

In [87]:
# 将等级索引series对象转换为简单的dataframe对象
data.unstack()

,ball,pen,pencil
black,3,4,5
red,6,7,8
white,0,1,2


##### 等级索引的DataFrame

In [96]:
# 对于dataframe对象，可以为它的行和列都定义为等级索引。声明dataframe对象时，为index选项和
# columns选项分别指定一个元素为数组的数组
df=pd.DataFrame(np.random.randn(16).reshape(4,4),
               index=[['white','white','red','red'],['up','down','up','down']],
               columns=[['pen','pen','paper','paper'],[1,2,1,2]])
df  # 多重索引的dataframe

pen               paper          
                   1         2         1         2
white up   -0.636249 -1.144386 -1.064231 -0.626650
      down  2.515040 -1.079589 -0.782443  0.003044
red   up    0.381698  1.489234 -0.767064  0.905798
      down -1.530355  3.265927  1.593545  0.758473

In [97]:
# 为等级dataframe添加列名和标签名
df.columns.names=['objects','id']
df.index.names=['colors','status']
df

objects             pen               paper          
id                    1         2         1         2
colors status                                        
white  up     -0.636249 -1.144386 -1.064231 -0.626650
       down    2.515040 -1.079589 -0.782443  0.003044
red    up      0.381698  1.489234 -0.767064  0.905798
       down   -1.530355  3.265927  1.593545  0.758473

In [98]:
# swaplevel()以要互换位置的两个层级的名称为参数，返回交换位置后的一个新对象，
# 其中各元素的顺序保存不变
df.swaplevel('colors','status')

objects             pen               paper          
id                    1         2         1         2
status colors                                        
up     white  -0.636249 -1.144386 -1.064231 -0.626650
down   white   2.515040 -1.079589 -0.782443  0.003044
up     red     0.381698  1.489234 -0.767064  0.905798
down   red    -1.530355  3.265927  1.593545  0.758473

In [94]:
# sortlevel():只根据一个层级对数据排序
df.sort_index(level='colors')

objects             pen               paper          
id                    1         2         1         2
colors status                                        
red    down    0.318608  0.267701 -0.624243  0.169180
       up     -0.466779 -1.762188  0.814863  0.137036
white  down    0.093938  0.242724  1.566758 -0.829444
       up     -0.190680 -1.071839 -0.747329  0.468296

In [101]:
df.sort_index(0,ascending=False)# 以行按第一层进行排序

objects             pen               paper          
id                    1         2         1         2
colors status                                        
white  up     -0.190680 -1.071839 -0.747329  0.468296
       down    0.093938  0.242724  1.566758 -0.829444
red    up     -0.466779 -1.762188  0.814863  0.137036
       down    0.318608  0.267701 -0.624243  0.169180

In [103]:
df.sort_index(0,ascending=True)  #若想对列进行操作，axis=1

objects             pen               paper          
id                    1         2         1         2
colors status                                        
red    down    0.318608  0.267701 -0.624243  0.169180
       up     -0.466779 -1.762188  0.814863  0.137036
white  down    0.093938  0.242724  1.566758 -0.829444
       up     -0.190680 -1.071839 -0.747329  0.468296

In [104]:
# 多层次索引的数据，汇总时可按照级别进行
df.sum(level='colors')

objects       pen               paper          
id              1         2         1         2
colors                                         
white   -0.096742 -0.829115  0.819428 -0.361148
red     -0.148171 -1.494487  0.190619  0.306216

In [105]:
df.sum(level='status')

objects       pen               paper          
id              1         2         1         2
status                                         
up      -0.657459 -2.834027  0.067533  0.605332
down     0.412546  0.510426  0.942514 -0.660265

In [106]:
df.sum(level='objects',axis=1)

objects             pen     paper
colors status                    
white  up     -1.262520 -0.279033
       down    0.336663  0.737313
red    up     -2.228966  0.951899
       down    0.586309 -0.455064

参考：https://blog.csdn.net/ialexanderi/article/details/78614967

### 1.5 删除 (即drop行筛选、del列筛选或drop（axis=1))
删除多余的列和行

In [60]:
import pandas as pd
import numpy as np
frame3=pd.DataFrame(np.arange(9).reshape(3,3),
                    index=['white','black','red'],
                   columns=['ball','pen','pencil'])
frame3

,ball,pen,pencil
white,0,1,2
black,3,4,5
red,6,7,8


In [61]:
# 删除一列，对dataframe对象应用del命令，指定列名
del frame3['pen']
frame3

,ball,pencil
white,0,2
black,3,5
red,6,8


In [62]:
# 删除多余的行，使用drop，将索引的名称作为参数
frame3.drop('red')
# 删除多余的列
# frame3.drop（‘columnname’，axis=1）

,ball,pencil
white,0,2
black,3,5


## 2、数据转换

### 2.1 删除重复元素drop_duplicated()（数据去重）

In [8]:
frame=pd.DataFrame({'color':['white','red','white','red','white'],
                   'value':[2,1,2,1,3]})
frame

,color,value
0,white,2
1,red,1
2,white,2
3,red,1
4,white,3


dataframe对象的duplicated（）函数可用来检测重复的行，返回元素为布尔型的series对象。每个元素对应一行，如果该行与其他行重复（也就是说该行不是第一次出现），则元素为true，如果跟前面不重复，则元素就为false

#### duplicated()：查看dataframe对象中是否含有重复项

In [9]:
frame.duplicated()

0    False
1    False
2     True
3     True
4    False
dtype: bool

#### 返回元素为布尔值的series对象用处很大，特别适用于过滤操作

In [11]:
frame[frame.duplicated()]  # 输出布尔值的series对象中true的项。即重复的项

,color,value
2,white,2
3,red,1


#### drop_duplicates():将dataframe对象中所有重复的行删除，返回的是删除重复行后的dataframe对象

In [14]:
frame5=pd.DataFrame({'color':['white','red','white','red','white'],
                   'value':[2,1,2,1,3]})
frame5.drop_duplicates()

,color,value
0,white,2
1,red,1
4,white,3


### 2.2 异常值检测和过滤  any( )
any（1）对每一列应用筛选条件

In [16]:
# 创建三列的dataframe对象，每一列都包含1000个随机数
randframe=pd.DataFrame(np.random.randn(1000,3))
randframe.describe()

,0,1,2
count,1000.000000,1000.000000,1000.000000
mean,-0.018778,-0.016878,0.009964
std,1.022229,0.991302,1.011044
min,-3.138252,-3.121206,-2.826183
25%,-0.693749,-0.635697,-0.686369
50%,-0.015156,-0.029350,0.024944
75%,0.715835,0.624838,0.694032
max,3.129515,3.026632,2.974986


#### 将比标准差大3倍的元素视作异常值，用std()函数就可以求得dataframe对象每一列的标准差

In [17]:
randframe.std()

0    1.022229
1    0.991302
2    1.011044
dtype: float64

#### 借助any（）函数，根据每一列的标准差，对dataframe对象的所有元素进行过滤

In [19]:
# any(1)对每一列应用筛选条件；>3*std得出异常值，<3*std得出正常值
randframe[(np.abs(randframe)>(3*randframe.std())).any(1)]

,0,1,2
435,0.071949,-3.121206,0.549659
912,3.129515,0.705749,-0.110397
917,-3.138252,1.402405,-0.403070
998,0.105272,3.026632,0.143040


### 2.3 排序  take()  vs. sort()（即数据排序）

### 2.3.1 随机排序（对行或列进行排序）

#### 用permutation（）函数创建一个包含0-4这五个整数的数组,按照这个数组元组的顺序为dataframe对象的行排序
permutation():调整series和dataframe对象各行的顺序

In [6]:
frame6=pd.DataFrame(np.arange(25).reshape(5,5))
frame6

,0,1,2,3,4
0,0,1,2,3,4
1,5,6,7,8,9
2,10,11,12,13,14
3,15,16,17,18,19
4,20,21,22,23,24


In [7]:
new_order=np.random.permutation(5)
new_order

array([4, 3, 0, 2, 1])

#### 行随机排列：对dataframe对象的所有行应用take（）函数，把新的次序传给它

In [31]:
# 行随机排列
frame6.take(new_order)

,0,1,2,3,4
4,20,21,22,23,24
3,15,16,17,18,19
0,0,1,2,3,4
2,10,11,12,13,14
1,5,6,7,8,9


In [32]:
# 列随机排列
frame6.take(new_order,axis=1)

,4,3,0,2,1
0,4,3,0,2,1
1,9,8,5,7,6
2,14,13,10,12,11
3,19,18,15,17,16
4,24,23,20,22,21


### 2.3.2、升降排序sort()（对行或列中的数据进行排序）

###### 对单列数据进行排序

In [23]:
# 对单列数据进行排序
frame7=pd.DataFrame({'id':['ball','pencil','pen','mug','ashtray'],
                    'price':[12.22,31.20,33.31,10.30,23.00],
                    'brand':['OME','DE','LV','ER','DQ'],
                     'income':[7.44,26.34,23.20,10.23,12.33],
                    'cost':[5.33,5.33,9.22,1.22,11.34]})
frame7

,brand,cost,id,income,price
0,OME,5.33,ball,7.44,12.22
1,DE,5.33,pencil,26.34,31.20
2,LV,9.22,pen,23.20,33.31
3,ER,1.22,mug,10.23,10.30
4,DQ,11.34,ashtray,12.33,23.00


In [24]:
frame7.sort_values(['income'],ascending=True)

,brand,cost,id,income,price
0,OME,5.33,ball,7.44,12.22
3,ER,1.22,mug,10.23,10.30
4,DQ,11.34,ashtray,12.33,23.00
2,LV,9.22,pen,23.20,33.31
1,DE,5.33,pencil,26.34,31.20


##### ascending=True：升序；ascending=False:降序

###### 对多列数据进行排序

In [27]:
# 对多列数据进行排序
frame7.sort_values(['id','cost'],ascending=False) 
#先对id进行降序，若遇到相同的id，则对cost降序

,brand,cost,id,income,price
1,DE,5.33,pencil,26.34,31.20
2,LV,9.22,pen,23.20,33.31
3,ER,1.22,mug,10.23,10.30
0,OME,5.33,ball,7.44,12.22
4,DQ,11.34,ashtray,12.33,23.00


In [28]:
frame7.sort_values(['cost','id'],ascending=False)
# 先对cost降序，若遇到相同的cost，则对id降序

,brand,cost,id,income,price
4,DQ,11.34,ashtray,12.33,23.00
2,LV,9.22,pen,23.20,33.31
1,DE,5.33,pencil,26.34,31.20
0,OME,5.33,ball,7.44,12.22
3,ER,1.22,mug,10.23,10.30


### 获取排序后的前几列（即TopN筛选）

In [29]:
# 最小前几项
frame7.sort_values(['price'],ascending=True).head(3)

,brand,cost,id,income,price
3,ER,1.22,mug,10.23,10.30
0,OME,5.33,ball,7.44,12.22
4,DQ,11.34,ashtray,12.33,23.00


In [30]:
# 最大前几项
frame7.sort_values(['price'],ascending=False).head(3)

,brand,cost,id,income,price
2,LV,9.22,pen,23.20,33.31
1,DE,5.33,pencil,26.34,31.20
4,DQ,11.34,ashtray,12.33,23.00


### 2.4 映射（即数据值替换、列追加、列名修改）
映射关系：创建一个映射关系列表，把元素跟一个特定的标签或字符串绑定起来

映射关系对象最好为字典dict（）

#### 2.4.1 用映射替换元素 replace( )（即数据值替换）

In [47]:
frame=pd.DataFrame({'item':['ball','mug','pen','pencil','ashtray'],
                   'color':['white','rosso','verde','black','yellow'],
                   'price':[5.56,4.20,1.30,0.56,2.75]})
frame

,color,item,price
0,white,ball,5.56
1,rosso,mug,4.20
2,verde,pen,1.30
3,black,pencil,0.56
4,yellow,ashtray,2.75


##### 用新元素替换不确定的元素，需要定义一组映射关系。在映射关系中，旧元素作为键，新元素作为值

In [48]:
# 建立映射关系
newcolors={'rosso':'red',
          'verde':'green'}
# 调用replace函数，传入表示映射关系的字典作为参数
frame.replace(newcolors)

,color,item,price
0,white,ball,5.56
1,red,mug,4.20
2,green,pen,1.30
3,black,pencil,0.56
4,yellow,ashtray,2.75


In [49]:
# NaN替换为0
ser=pd.Series([1,3,np.nan,4,5,np.nan])
ser

0    1.0
1    3.0
2    NaN
3    4.0
4    5.0
5    NaN
dtype: float64

In [50]:
ser.replace(np.nan,0)

0    1.0
1    3.0
2    0.0
3    4.0
4    5.0
5    0.0
dtype: float64

#### 2.4.2 用映射添加元素 map（）（即列追加）

 利用映射关系从另外一个数据结构获取元素，将其添加到目标数据结构的列中。映射对象总是要单独定义的

In [44]:
frame=pd.DataFrame({'item':['ball','mug','pen','pencil','ashtray'],
                   'color':['white','rosso','verde','black','yellow']})
frame

,color,item
0,white,ball
1,rosso,mug
2,verde,pen
3,black,pencil
4,yellow,ashtray


In [46]:
# 假定有一份价格清单，记录了每种商品的价格，添加price这一列。先定义一个dict对象
price={'ball':5.56,
      'mug':3.20,
      'pen':1.20,
      'pencil':2.30,
      'ashtray':2.76}
# map()函数可应用于series或dataframe对象的一列，它接收一个函数或表示映射关系的字典对象作为参数
frame['price']=frame['item'].map(price)
frame

,color,item,price
0,white,ball,5.56
1,rosso,mug,3.20
2,verde,pen,1.20
3,black,pencil,2.30
4,yellow,ashtray,2.76


在dataframe的item这一列应用映射关系，用字典price作为参数，为dataframe对象添加price列

##### 用映射添加元素：从A数据结构获取元素，并添加到目标数据结构（B）的列中：frame['B_p'].map(A):表示在B的‘p’列应用映射关系，对B添加字典A列

#### 2.4.3 重命名轴索引rename()（即列名修改）

In [53]:
# 修改轴标签
reindex={0:'first',
        1:'second',
        2:'third',
        3:'fourth',
        4:'fifth'}
frame.rename(reindex)

,color,item,price
first,white,ball,5.56
second,rosso,mug,4.20
third,verde,pen,1.30
fourth,black,pencil,0.56
fifth,yellow,ashtray,2.75


In [56]:
# 修改列名称
recolumn={'item':'object',
         'price':'value'}
frame.rename(index=reindex,columns=recolumn)

,color,object,value
first,white,ball,5.56
second,rosso,mug,4.20
third,verde,pen,1.30
fourth,black,pencil,0.56
fifth,yellow,ashtray,2.75


In [58]:
# 修改单个元素的列名和标签名
frame.rename(index={1:'frist'},columns={'item':'object'})

,color,object,price
0,white,ball,5.56
frist,rosso,mug,4.20
2,verde,pen,1.30
3,black,pencil,0.56
4,yellow,ashtray,2.75


### 2.5 离散化和面元划分 cut()(即数据离散化)
连续数据被离散化或拆分为‘面元’（bin），这个过程要用到cut()

cut(x,bin,right=True,labels=None)：x为被切对象，应当是一维的类数组结构；bin：可以是序列、整数或标量

序列：按序列的元素间隔划分x，返回x各个元素的分组情况

In [64]:
# 定义一个数组，存储用于面元划分的各数值
bins=[0,3,6,9]
ser=pd.Series(np.random.randint(1,10,6))
ser

0    9
1    1
2    9
3    8
4    3
5    4
dtype: int32

random.randint(a,b,n):用于生成一个指定范围内的整数，a为下限，b为上限，生成n各随机数（b≥n≥a）

random.random（）:用于生成一个0到1的随机浮点数:0≤n≤1.0

random.shuffle(x[,random]):用于将一个列表中的元素打乱

random.sample(sequence,k):从指定序列中随机获取指定长度的片段，sample函数不会修改原有序列

In [65]:
cats=pd.cut(ser,bins,labels=['small','middle','large'])
cats

0     large
1     small
2     large
3     large
4     small
5    middle
dtype: category
Categories (3, object): [small < middle < large]

In [66]:
pd.value_counts(cats)

large     3
small     2
middle    1
dtype: int64

##### qcut( ):根据样本的分位数对数据进行面元划分

In [68]:
ser=np.random.randint(0,100,1000)
cats=pd.qcut(ser,10)
pd.value_counts(cats)

(9.0, 19.0]      106
(39.0, 49.0]     105
(-0.001, 9.0]    105
(28.7, 39.0]     103
(57.4, 68.0]     102
(68.0, 79.0]     101
(89.0, 99.0]      99
(79.0, 89.0]      98
(49.0, 57.4]      92
(19.0, 28.7]      89
dtype: int64

### 2.6 随机取样（即数据抽样）
基本思路：先利用np.random模块随机生成一个需要的索引，然后利用这个索引去源数据里过滤取值

In [69]:
# np.random.randint(start,end,size):一般用于实现“可重取”的随机抽样，因为返回的数组中的元素可重复，而且size可变
frame8=pd.DataFrame(np.arange(25).reshape(5,5))
sample=np.random.randint(0,len(frame8),size=3)
sample

array([4, 1, 0])

In [70]:
frame8.take(sample)

,0,1,2,3,4
4,20,21,22,23,24
1,5,6,7,8,9
0,0,1,2,3,4


In [73]:
frame8[sample]

,4,1,0
0,4,1,0
1,9,6,5
2,14,11,10
3,19,16,15
4,24,21,20


### 2.7 字符串处理（即列拆分）

#### 6.6.1 内置的字符串处理方法  split( )(即列拆分)
将复合字符串分为几部分，分别赋给不同的变量。split():以参考点为分隔符，将文本分为几部分

In [75]:
text='16 Bolton Avenue   , Boston'
text.split(',')

['16 Bolton Avenue   ', ' Boston']

##### strip()删除多余的空白字符（包括换行符）

In [78]:
tokens=[s.strip() for s in text.split(',')]
tokens

['16 Bolton Avenue', 'Boston']

##### 拆分列名赋值

In [80]:
address,city=[s.strip() for s in text.split(',')]
address

'16 Bolton Avenue'

In [81]:
city

'Boston'

##### 字符串拼接  join( )
将连接符作为对象调用join函数

In [83]:
strings=['A+','a-','B','BB-','C+']
';'.join(strings)

'A+;a-;B;BB-;C+'

##### 查找字符串  in  vs.  index  vs.  find

In [84]:
'Boston' in text

True

In [85]:
text.index('Boston')

21

In [86]:
text.find('Boston')

21

##### 计算字符出现的次数   count（）

In [87]:
text.count('e')

2

In [88]:
text.count('Avenue')

1

##### 替换或删除字符串 replace()

In [89]:
text.replace('Avenue','Street')

'16 Bolton Street   , Boston'

In [92]:
text.replace('16','')  # 用空值符替换子串，效果等于删除子串

' Bolton Avenue   , Boston'

#### 6.6.2 正则表达式
正则表达式：用事先定义好的一些特定字符，及这些特定字符的组合，组成一个‘规则字符串’，这个‘规则字符串’用来表达对字符串的一种过滤逻辑。

单条正则表达式通常被称为regex。Python内置的re模块用于操作regex对象

目的：给定一个正则表达式和另一个字符串，可以达到的目的

1、给定的字符串是否符合正则表达式的过滤逻辑（称为‘匹配’）

2、可以通过正则表达式，从字符串中获取我们想要的特定部分

##### 只有先导入re模块，才能使用正则表达式

In [139]:
import re
# 模式匹配；替换；切分

##### 例子：表示一个或多个空白字符的正则表达式为\s+。re模块同样有文本分隔函数split()

##### 方法1：直接调用re模块

In [141]:
text='This is     an\t odd \n test'
re.split('\s+',text)
# 调用re.split()函数时，先编辑正则表达式，然后在作为参数传入的文本上调用split（）函数

['This', 'is', 'an', 'odd', 'test']

##### 方法2：预测编译正则表达式

In [144]:
regex=re.compile('\s+') # 创建regex对象后直接调用split（）

In [145]:
regex.split(text)

['This', 'is', 'an', 'odd', 'test']

##### findall():匹配文本中所有符合正则表达式的子串，该函数返回一个列表，元素为文本所有符合正则表达式的子串

In [149]:
text='This is my an apple. It belongs to me. I have an Apd'
re.findall('[A,a]\w+',text)  # 查找text以a和A开头的单词

['an', 'apple', 'ave', 'an', 'Apd']

In [154]:
# 返回子串在字符串中开始和结束的位置
search=re.search('[a,A]\w+',text)

In [152]:
search.start()

11

In [153]:
search.end()

13

## 3、数据聚合（即行列交叉统计）
先把数据分成几组，再为不同的数据应用不同的函数以转换数据。转换数据，使每一个数组生成一个单一的数值。

分组和应用函数这两个阶段经常用一步来完成

### GroupBy（split-apply-combine 分组-用函数处理-合并结果）。

分组：将数据集分成多个组（既可以应用于行（axis=0），也可以与列（axis=1））

用函数处理：用函数处理每一组

合并：把不同组得到的结果合并起来

In [155]:
frame=pd.DataFrame({'color':['white','white','red','green','red'],
                   'object':['pen','pencil','pencil','pen','ashtray'],
                   'price1':[5.34,4.22,1.30,0.56,2.75],
                   'price2':[4.75,4.23,1.60,0.75,3.15]})
frame

,color,object,price1,price2
0,white,pen,5.34,4.75
1,white,pencil,4.22,4.23
2,red,pencil,1.30,1.60
3,green,pen,0.56,0.75
4,red,ashtray,2.75,3.15


##### 将列color作为组标签，计算price1的均值

In [105]:
# 首先将含有相同颜色的行分到同一个组里（即分组）
group=frame['price1'].groupby(frame['color'])
group.groups

{'green': Int64Index([3], dtype='int64'),
 'red': Int64Index([2, 4], dtype='int64'),
 'white': Int64Index([0, 1], dtype='int64')}

In [107]:
# 其次将分组后的元素进行计算并且合并
group.mean()

color
green    0.560
red      2.025
white    4.780
Name: price1, dtype: float64

### 3.1 等级分组
使用多个键，按照等级关系分组

In [109]:
group1=frame['price1'].groupby([frame['color'],frame['object']])
group1.groups

{('green', 'pen'): Int64Index([3], dtype='int64'),
 ('red', 'ashtray'): Int64Index([4], dtype='int64'),
 ('red', 'pencil'): Int64Index([2], dtype='int64'),
 ('white', 'pen'): Int64Index([0], dtype='int64'),
 ('white', 'pencil'): Int64Index([1], dtype='int64')}

In [110]:
group1.sum()

color  object 
green  pen        0.56
red    ashtray    2.75
       pencil     1.30
white  pen        5.34
       pencil     4.22
Name: price1, dtype: float64

In [114]:
frame[['price1','price2']].groupby([frame['object']]).sum()

,price1,price2
object,,
ashtray,2.75,3.15
pen,5.90,5.50
pencil,5.52,5.83


In [116]:
frame[['price1','price2']].groupby([frame['object']]).mean()

,price1,price2
object,,
ashtray,2.75,3.150
pen,2.95,2.750
pencil,2.76,2.915


#### 插播
count：分组中非nan值得数量；sum：非na值的和；mean：非na值的平均值；median：非na值的算术中位数；std，var：无偏（分母为n-1）标准差和方差；min，max：非na值的最小值和最大值；prod：非na值的积；fist、last：第一个和最后一个非na值

### 3.2 组迭代
生成一系列由各组名称及其数据部分组成的元组

In [120]:
# 按颜色对整个dataframe进行分组
for name,group in frame.groupby('color'):
    print (name)
    print(group)

green
   color object  price1  price2
3  green    pen    0.56    0.75
red
  color   object  price1  price2
2   red   pencil    1.30    1.60
4   red  ashtray    2.75    3.15
white
   color  object  price1  price2
0  white     pen    5.34    4.75
1  white  pencil    4.22    4.23


#### 3.2.1 链式转换

In [122]:
frame['price1'].groupby(frame['color']).mean()

color
green    0.560
red      2.025
white    4.780
Name: price1, dtype: float64

In [123]:
frame.groupby(frame['color'])['price1'].mean()

color
green    0.560
red      2.025
white    4.780
Name: price1, dtype: float64

In [124]:
(frame.groupby(frame['color']).mean())['price1']

color
green    0.560
red      2.025
white    4.780
Name: price1, dtype: float64

##### 在列名前添加前缀  add_prefix('a_')

In [127]:
frame.groupby('color').mean().add_prefix('mean_')

,mean_price1,mean_price2
color,,
green,0.560,0.750
red,2.025,2.375
white,4.780,4.490


#### 3.2.2 分组函数

In [134]:
# 计算分位数 quantile()
group=frame.groupby('color')
group['price1'].quantile(0.6)

color
green    0.560
red      2.170
white    4.892
Name: price1, dtype: float64

##### 自定义聚合函数 agg（func）

In [135]:
def range(series):
    return series.max()-series.min()

In [136]:
group['price1'].agg(range)

color
green    0.00
red      1.45
white    1.12
Name: price1, dtype: float64

In [137]:
# 对整个dataframe对象应用agg（）函数
group.agg(range)

,price1,price2
color,,
green,0.00,0.00
red,1.45,1.55
white,1.12,0.52


In [138]:
# 同时使用多个聚合函数
group['price1'].agg(['mean','std',range])

,mean,std,range
color,,,
green,0.560,NaN,0.00
red,2.025,1.025305,1.45
white,4.780,0.791960,1.12


### 3.3 高级数据聚合  transform（）& apply（）

In [156]:
frame.groupby('color').transform(np.sum).add_prefix('tot_')

,tot_object,tot_price1,tot_price2
0,penpencil,9.56,8.98
1,penpencil,9.56,8.98
2,pencilashtray,4.05,4.75
3,pen,0.56,0.75
4,pencilashtray,4.05,4.75


##### apply():把对象分为几部分后，再用函数处理每一部分，各步骤之间用链式方法连接在一起

In [166]:
group=frame.groupby(['color','object']).apply(lambda x:x.max())
group

color   object  price1  price2
color object                                 
green pen      green      pen    0.56    0.75
red   ashtray    red  ashtray    2.75    3.15
      pencil     red   pencil    1.30    1.60
white pen      white      pen    5.34    4.75
      pencil   white   pencil    4.22    4.23